In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import nltk
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
import pandas as pd

nltk.download('vader_lexicon', quiet=False)

vader_model = SentimentIntensityAnalyzer()



/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
data = pd.read_csv(r'/content/746 songs.csv', error_bad_lines=False);
data_text = data[['lyrics']]
documents = data_text
lyrics_list = documents['lyrics'].tolist()

'''for sentences in lyrics_list:
    scores = vader_model.polarity_scores(sentences)
    print()
    print('INPUT SENTENCE', sentences)
    print('VADER OUTPUT', scores)'''

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


"for sentences in lyrics_list:\n    scores = vader_model.polarity_scores(sentences)\n    print()\n    print('INPUT SENTENCE', sentences)\n    print('VADER OUTPUT', scores)"

In [ ]:
nlp = spacy.load('en')

In [ ]:
def vader_output_to_label(vader_output):
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    elif compound > 0.0:
        return 'positive'
    
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.0}) == 'neutral'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.01}) == 'positive'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': -0.01}) == 'negative'

In [ ]:
def run_vader(textual_unit, 
              lemmatize=False, 
              parts_of_speech_to_consider=None,
              verbose=0):
    """
    Run VADER on a sentence from spacy
    
    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -None or empty set: all parts of speech are provided
    -non-empty set: only these parts of speech are considered.
    :param int verbose: if set to 1, information is printed
    about input and output
    
    :rtype: dict
    :return: vader output dict
    """
    documents = nlp(textual_unit)
        
    input_to_vader = []

    for sent in documents.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-': 
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add) 
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))
    
    if verbose >= 1:
        print()
        print('INPUT SENTENCE', sent)
        print('INPUT TO VADER', input_to_vader)
        print('VADER OUTPUT', scores)

    return scores

In [ ]:
all_vader_output = []

for lyr in lyrics_list:
  vader_info = run_vader(lyr, lemmatize=True,verbose = 1)
  vader_label = vader_output_to_label(vader_info)
  all_vader_output.append(vader_label)

print(all_vader_output)




INPUT SENTENCE Under someone's bed
You put me on and said I was your favorite142EmbedShare URLCopyEmbedCopy
INPUT TO VADER ['Vintage', 'tee', ',', 'brand', 'new', 'phone', '\n', 'high', 'heel', 'on', 'cobblestone', '\n', 'when', 'you', 'be', 'young', ',', 'they', 'assume', 'you', 'know', '\u2005', 'nothing', '\n', 'Sequin', '\u2005', 'smile', ',', 'black', 'lipstick', '\n', 'Sensual', '\u2005', 'politic', '\n', 'when', 'you', 'be', 'young', ',', 'they', 'assume', '\u2005', 'you', 'know', 'nothing', '\n\n', 'but', 'I', 'know', 'you', '\n', 'Dancin', "'", 'in', 'your', 'Levi', "'s", '\n', 'Drunk', 'under', 'a', 'streetlight', ',', 'I', '\n', 'I', 'know', 'you', '\n', 'hand', 'under', 'my', 'sweatshirt', '\n', 'Baby', ',', 'kiss', 'it', 'better', ',', 'I', '\n\n', 'and', 'when', 'I', 'feel', 'like', 'I', 'be', 'an', 'old', 'cardigan', '\n', 'under', 'someone', "'s", 'bed', '\n', 'You', 'put', 'me', 'on', 'and', 'say', 'I', 'be', 'your', 'favorite', '\n\n', 'a', 'friend', 'to', 'all', 'be

In [ ]:
# Add sentiment column to dataset
final_doc = documents.assign(sentiment=all_vader_output)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(final_doc)
# Convert to .csv file
final_doc.to_csv('Sentiment analysis.csv',index = False)

                                                  lyrics  index sentiment
0      hold time feel break feel untrue convince spea...      0  positive
1      believe drop rain fall grow believe darkest ni...      1  negative
2      sweetheart send letter goodbye secret feel bet...      2  positive
3      kiss lips want stroll charm mambo chacha merin...      3  positive
4      till darling till matter know till dream live ...      4  positive
5      convoy light dead ahead merchantmen trump dies...      5  negative
6      piece mindin world knowin life come bring give...      6  positive
7      care moment hold fast press lips dream heaven ...      7  positive
8      lonely night surround power read mind hour nig...      8  positive
9      tear heart seat stay awhile tear heart game st...      9  negative
10     better sail midnight better dive pearl better ...     10  positive
11     moon shin shin wait moon pin pin place long ho...     11  negative
12     come motherfucker spit shit min